## **Import dependencies**

In [2]:
import pandas as pd    
import numpy as np
from sklearn.metrics import classification_report
from classifiers import build_tuned_nn, build_tuned_rfc, build_tuned_svc
# from get_data_from_androzoo import _extracted_from_check_permissions_in_manifest_

2023-05-30 07:10:53.220492: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 07:10:53.401267: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-30 07:10:53.402669: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 07:10:54.358959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### **Define list of permissions**

In [3]:
permissions = [
    'ACCESS_CHECKIN_PROPERTIES', 'ACCESS_COARSE_LOCATION', 'ACCESS_FINE_LOCATION', 'ACCESS_NOTIFICATIONS',
    'ACCESS_WIFI_STATE', 'ADD_VOICEMAIL', 'ANSWER_PHONE_CALLS', 'BLUETOOTH_MAP', 'BODY_SENSORS', 'BROADCAST_WAP_PUSH',
    'CALL_LOG', 'CALL_PHONE', 'CAMERA', 'CAPTURE_VIDEO_OUTPUT', 'CHANGE_NETWORK_STATE', 'CHANGE_WIFI_STATE',
    'DISABLE_KEY_GUARD', 'GET_ACCOUNTS', 'GET_TASKS', 'INSTALL_PACKAGES', 'INTERNET', 'MANAGE_DOCUMENTS',
    'PERSISTENT_ACTIVITY', 'PROCESS_OUTGOING_CALLS', 'READ_CALENDAR', 'READ_CALL_LOG', 'READ_CONTACTS',
    'READ_EXTERNAL_STORAGE', 'READ_HISTORY_BOOKMARKS', 'READ_LOGS', 'READ_PHONE_NUMBERS', 'READ_PHONE_STATE',
    'READ_SMS', 'READ_SYNC_SETTINGS', 'RECEIVE_BOOT_COMPLETED', 'RECEIVE_MMS', 'RECEIVE_SMS', 'RECEIVE_WAP_PUSH',
    'RECORD_AUDIO', 'RECOVERY', 'RESTART_PACKAGES', 'SEND_SMS', 'SET_ALWAYS_FINISH', 'SET_WALLPAPER',
    'SYSTEM_ALERT_WINDOW', 'USE_SIP', 'WRITE_APN_SETTINGS', 'WRITE_CALENDAR', 'WRITE_CALL_LOG', 'WRITE_CONTACTS',
    'WRITE_EXTERNAL_STORAGE', 'WRITE_SETTINGS']


### **Read a dataframe**

In [4]:
df = pd.read_csv('./csv_files/merged_df_with_dates.csv', index_col='SHA256')
attack_type = 'no_attack'
# attack_type = 'random_number'
# attack_type = 'gaussian_signal_noise'
# attack_type = 'signal_noise'
# attack_type = 'function_from_article'

#### Split dataset by date to simulate new malware to detect

In [5]:
df[['vt_scan_date', 'dex_date', 'added']] = df[['vt_scan_date', 'dex_date', 'added']].apply(lambda x: pd.to_datetime(x))

In [6]:
X_train, y_train = df[df['vt_scan_date'] <'2021-10-01'].drop(['is_malware'], axis=1), df[df['vt_scan_date']<'2021-10-01'].is_malware
X_test, y_test = df[df['vt_scan_date']>='2021-10-01'].drop(['is_malware'], axis=1), df[df['vt_scan_date']>='2021-10-01'].is_malware
X_test = X_test.select_dtypes(include=['number'])
X_train = X_train.select_dtypes(include=['number'])



### Implement attack type

In [7]:
if  attack_type == 'random_number':
    from backdoor_attacks import add_random_binary 
    from random import randint
    down, up = 1, 20
    X_train = X_train.apply(lambda row: add_random_binary(row, randint(down, up), axis=1, result_type='broadcast'))
 
elif attack_type == 'gaussian_signal_noise':
    from backdoor_attacks import add_noise, gaussian_noise  
    X_train = X_train.apply(lambda row: add_noise(row, gaussian_noise), axis=1)

elif attack_type == 'signal_noise':
    from backdoor_attacks import add_noise, uniform_noise  
    X_train = X_train.apply(lambda row: add_noise(row, uniform_noise), axis=1)

#### Run classifiers

In [8]:
from classifiers import param_grid_rfc, param_grid_svc
classifiers = [
    (build_tuned_nn, {'x_train': X_train, 'y_train': y_train}, 'Neural Network'),
    (build_tuned_rfc, {'x_train': X_train, 'y_train': y_train, 'param_grid': param_grid_rfc},'Random Forest'),
    (build_tuned_svc, {'x_train': X_train, 'y_train': y_train, 'param_grid': param_grid_svc}, 'SVM')]



# nn = build_tuned_nn(X_train, y_train)
# nn.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0)
#
# y_pred = np.round(nn.predict(X_test))
# report = classification_report(y_test, y_pred, output_dict=True)


In [9]:
reports = {}
for clf, params, name in classifiers:
    model, best_params = clf(**params)
    model.fit(X_train, y_train)
    y_pred = np.round(model.predict(X_test))
    reports[name] = classification_report(y_test, y_pred, output_dict=True)

Trial 5 Complete [00h 00m 59s]
val_accuracy: 0.959967315196991

Best val_accuracy So Far: 0.959967315196991
Total elapsed time: 00h 05m 01s
INFO:tensorflow:Oracle triggered exit
109/109 [==============================] - 0s 2ms/step


/home/bartek/PycharmProjects/android_bin_back/venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/bartek/PycharmProjects/android_bin_back/venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/bartek/PycharmProjects/android_bin_back/venv/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the pas

In [11]:
for res in reports.items():
    print(res)

('Neural Network', {'0': {'precision': 0.9376538146021329, 'recall': 0.9744245524296675, 'f1-score': 0.9556856187290971, 'support': 2346}, '1': {'precision': 0.9423631123919308, 'recall': 0.8658428949691086, 'f1-score': 0.9024839006439742, 'support': 1133}, 'accuracy': 0.9390629491233113, 'macro avg': {'precision': 0.9400084634970318, 'recall': 0.9201337236993881, 'f1-score': 0.9290847596865357, 'support': 3479}, 'weighted avg': {'precision': 0.9391874835862781, 'recall': 0.9390629491233113, 'f1-score': 0.9383595058833242, 'support': 3479}})
('Random Forest', {'0': {'precision': 0.9401853411962932, 'recall': 0.9514066496163683, 'f1-score': 0.9457627118644067, 'support': 2346}, '1': {'precision': 0.8968325791855204, 'recall': 0.8746690203000883, 'f1-score': 0.8856121537086684, 'support': 1133}, 'accuracy': 0.9264156366772061, 'macro avg': {'precision': 0.9185089601909068, 'recall': 0.9130378349582283, 'f1-score': 0.9156874327865376, 'support': 3479}, 'weighted avg': {'precision': 0.9260